## RAG 구현
- fine tuning 불편함 해결

In [1]:
!pip install chromadb -qqq # vector database

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 14.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 49.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0

In [2]:
import chromadb
chroma_client = chromadb.Client()
chroma_client

### Vector DB에 데이터 넣기

In [3]:
documents = [
    "대한민국의 수도인 서울은 한강을 중심으로 동서로 나뉘며, 다양한 문화와 역사가 공존하는 도시이다. 고궁과 현대식 빌딩이 조화를 이루며, 관광객들에게 많은 볼거리를 제공한다. 또한, 강남과 강북 지역의 차이도 서울의 독특한 특징 중 하나로 꼽힌다.",
    "한반도의 기후는 사계절이 뚜렷하며, 여름에는 북태평양 기단의 영향을 받아 무덥고 습하다. 반면 겨울에는 시베리아 기단의 영향을 받아 춥고 건조한 날씨가 지속된다. 봄과 가을은 짧지만 쾌적한 기후를 자랑하며, 여행하기 좋은 계절이다.",
    "김치는 한국을 대표하는 전통 음식으로, 배추나 무를 소금에 절여 양념을 발라 발효시킨다. 지역마다 다양한 종류가 있으며, 대표적으로 배추김치, 깍두기, 동치미 등이 있다. 한국인들은 김치를 주식과 함께 즐겨 먹으며, 건강에도 좋은 음식으로 알려져 있다.",
    "경복궁은 조선 시대에 건립된 궁궐로, 서울의 중심부에 위치해 있다. 광화문을 지나면 근정전과 경회루 등 다양한 건축물을 볼 수 있으며, 매년 많은 관광객들이 방문한다. 특히, 한복을 입고 방문하면 더욱 특별한 경험을 할 수 있다.",
    "제주도는 대한민국에서 가장 큰 섬으로, 독특한 자연환경과 관광 명소로 유명하다. 한라산을 비롯하여 성산일출봉, 용두암 등의 자연경관이 있으며, 다양한 전통 문화와 먹거리를 즐길 수 있다. 온화한 기후 덕분에 사계절 내내 여행하기 좋다."
]

In [4]:
# Vector data 를 영구성 있게 저장
chroma_client = chromadb.PersistentClient(path = './chroma_db')
chroma_client

In [7]:
from transformers import AutoTokenizer, AutoModel
# 모델 및 토크나이저 로드
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [8]:
## Embedding 과정 
text = documents[0]
inputs = tokenizer(text, return_tensors='pt'
             , truncation = True
             , padding = True
             , max_length = 512) # 사전 맞는 글자 나옴
# inputs

{'input_ids': tensor([[  101,  1457, 30007, 30005, 30006, 30021, 29995, 30019, 30021, 29991,
         30014, 30020, 29999, 30018,  1461, 30014, 29993, 30011, 29999, 30019,
         30021,  1461, 30008, 29999, 30014, 30022, 29999, 30017, 30021,  1469,
         30006, 30021, 29991, 30006, 30025, 29999, 30017, 30022,  1464, 30014,
         30025, 29997, 30019, 30023, 29999, 30017, 29994, 30011,  1457, 30011,
         30025, 29997, 30008, 29994, 30011,   100,  1010,   100,  1459, 30014,
         30021, 30005, 30012, 29999, 30012,  1463, 30010, 30020, 29997, 30006,
         29991, 30006,  1455, 30011, 30025, 30000, 30011, 30021, 30005, 30006,
         29992, 30017, 30021,  1457, 30011, 29997, 30019, 29999, 30019, 29993,
         30006,  1012,  1455, 30011, 29991, 30014, 30025, 29991, 30012,  1469,
         30010, 30021, 29993, 30007, 29997, 30019, 30020,  1460, 30019, 30022,
         29993, 30019, 30025, 29999, 30019,  1464, 30011, 30005, 30012, 29994,
         30017, 30022,  1463, 30019, 2

In [11]:
import torch
with torch.no_grad():
    embedding = model(**inputs).last_hidden_state[:, 0, :].squeeze().numpy()
embedding

array([-8.26943964e-02,  2.83078313e-01,  3.51984680e-01, -1.21131420e-01,
       -7.63268247e-02, -2.16001049e-01, -8.87172893e-02, -1.51869819e-01,
        2.39160046e-01, -1.54747665e-01,  3.31682026e-01,  2.06611544e-01,
        1.15314230e-01, -2.29622766e-01,  2.54395455e-01, -1.81165293e-01,
       -1.52802765e-01,  1.28980413e-01,  1.10485390e-01,  3.74625891e-01,
        1.04376696e-01,  1.49215132e-01, -1.98121537e-02, -1.79707542e-01,
       -2.39626333e-01,  1.85221791e-01, -7.01640323e-02, -4.32608277e-02,
        3.11028630e-01, -9.39681292e-01, -2.42199838e-01, -5.87984174e-02,
       -3.17703672e-02,  4.13709357e-02, -1.71108916e-01, -1.78418070e-01,
       -5.14832847e-02,  3.31123263e-01,  4.81580980e-02,  1.76339567e-01,
       -2.13059127e-01, -3.12879652e-01,  1.18538618e-01, -2.39916325e-01,
        9.15382579e-02,  4.53747213e-02, -5.03587127e-02, -3.18319589e-01,
       -2.87917376e-01, -4.31965687e-04, -1.72749564e-01,  1.21962830e-01,
       -1.32388622e-01, -

In [14]:
# Vector data에 data 넣기
collection = chroma_client.get_or_create_collection(name = 'semantic_cache') # Vector DB에 Collection name 부여

doc_id = 90
collection.add(documents = [text]
              , embeddings = [embedding.tolist()]
              , ids = [str(doc_id)] )

In [15]:
def embedded_text(text):
    inputs = tokenizer(text, return_tensors='pt'
                 , truncation = True, padding = True, max_length = 512) # 사전 맞는 글자 나옴
    with torch.no_grad():
        embedding = model(**inputs).last_hidden_state[:, 0, :].squeeze().numpy()
    return embedding.tolist()
    # pass

In [17]:
def add_vectors_to_chroma(text, doc_id):
    collection.add(documents = [text]
              , embeddings = [embedded_text(text)]
              , ids = [str(doc_id)])
    # pass

In [24]:
## documents를 chroma 저장 
for idx, document in enumerate(documents):
    add_vectors_to_chroma(document, idx)
    pass

In [27]:
## chroma 저장 확인
records = collection.get()
records
collection.get(include=['embeddings', 'documents'])

{'ids': ['90', '0', '1', '2', '3', '4'],
 'embeddings': array([[-0.0826944 ,  0.28307831,  0.35198468, ...,  0.41575274,
         -0.2411796 , -0.27682897],
        [-0.0826944 ,  0.28307831,  0.35198468, ...,  0.41575274,
         -0.2411796 , -0.27682897],
        [-0.0150467 ,  0.3318426 ,  0.45812592, ...,  0.22800536,
         -0.12948722, -0.15413682],
        [-0.10022879,  0.40820158,  0.3926912 , ...,  0.21185279,
         -0.17258941, -0.1847412 ],
        [-0.0715006 ,  0.41899884,  0.47206962, ...,  0.31835678,
         -0.24478345, -0.24589416],
        [-0.10568349,  0.43485618,  0.4710961 , ...,  0.31788516,
         -0.20172018, -0.17516159]]),
 'documents': ['대한민국의 수도인 서울은 한강을 중심으로 동서로 나뉘며, 다양한 문화와 역사가 공존하는 도시이다. 고궁과 현대식 빌딩이 조화를 이루며, 관광객들에게 많은 볼거리를 제공한다. 또한, 강남과 강북 지역의 차이도 서울의 독특한 특징 중 하나로 꼽힌다.',
  '대한민국의 수도인 서울은 한강을 중심으로 동서로 나뉘며, 다양한 문화와 역사가 공존하는 도시이다. 고궁과 현대식 빌딩이 조화를 이루며, 관광객들에게 많은 볼거리를 제공한다. 또한, 강남과 강북 지역의 차이도 서울의 독특한 특징 중 하나로 꼽힌다.',
  '한반도의 기후는 사계절이 뚜렷하며, 여름에는 북태평양

In [28]:
question = "북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?"
response = collection.query(query_embeddings=[embedded_text(question)], n_results=1)
print(f"질문: {question}\n결과: {response}\n")

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
결과: {'ids': [['1']], 'embeddings': None, 'documents': [['한반도의 기후는 사계절이 뚜렷하며, 여름에는 북태평양 기단의 영향을 받아 무덥고 습하다. 반면 겨울에는 시베리아 기단의 영향을 받아 춥고 건조한 날씨가 지속된다. 봄과 가을은 짧지만 쾌적한 기후를 자랑하며, 여행하기 좋은 계절이다.']], 'uris': None, 'data': None, 'metadatas': [[None]], 'distances': [[4.638633358663138]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}

